In [203]:
#1-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_1.csv')
#print(df.info())
#print(df.head())

#1-1-1
df1 = df.groupby('대륙')['맥주소비량'].mean()
df1_max = df1.idxmax()
#print(df1_max)

#1-1-2
df2 = df.loc[df['대륙'] == df1_max, ['국가', '맥주소비량']].sort_values(by = '맥주소비량', ascending = False).reset_index()
df2_max = df2.iloc[4, 1]
#print(df2_max)

#1-1-3
df3 = df.loc[df['국가'] == df2_max, '맥주소비량'].mean()
print(round(df3, 0))


#1-2
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_2.csv')
#print(df.info())
#print(df.head())

#1-2-1
df1 = df.drop('국가', axis=1)
df1_sum = df1.sum(axis=1)
df_gwan = df['관광']
df['관광객비율'] = round((df_gwan/df1_sum), 3)

df_sorted = df.sort_values(by='관광객비율', ascending = False).reset_index()
df_sorted_gwan = df_sorted.iloc[1, 2]
#print(df_sorted_gwan)

#1-2-2
df_sorted2 = df.sort_values(by='관광', ascending = False).reset_index()
df_sorted2_country = df_sorted2.iloc[1, 1]
result = df.loc[df['국가'] == df_sorted2_country, '공무'].mean()
#print(result)
#1-2-3
print(round(df_sorted_gwan + result, 0))

#1-3
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_3.csv')
#print(df.info())
#print(df.head())
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

df_after = df.loc[:, ['CO(GT)', 'NMHC(GT)']]
df_after_scaled = mms.fit_transform(df_after)
co_std = df_after_scaled[:, 0].std().round(2)
nmhc_std = df_after_scaled[:, 1].std().round(2)
print(co_std, nmhc_std)

247.0
8325.0
0.37 0.15


In [208]:
#2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_2_test.csv')

#2-1 (인포)

#print(train.info())
#print(test.info())

#2-2 (독-종 분리)
test_id = test['ID']
train_X = train.drop(['count'], axis=1)
train_y = train['count']
test_X = test.drop(['count'], axis=1)
test_y = test['count']

#2-3 (검증 데이터 생성)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=1)
print(train_X.shape, valid_X.shape, train_y.shape, valid_y.shape)

#2-4 (cat - num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

#2-5 (전처리)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown='ignore')
stdscaler = StandardScaler()

train_X_ohe = onehotencoder.fit_transform(train_X[cat_columns])
valid_X_ohe = onehotencoder.transform(valid_X[cat_columns])
test_X_ohe = onehotencoder.transform(test_X[cat_columns])

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test_X[num_columns])

train_X_preprocessed = np.concatenate([train_X_ohe, train_X_std], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_ohe, valid_X_std], axis=1)
test_X_preprocessed = np.concatenate([test_X_ohe, test_X_std], axis=1)

#2-6 (모델적합)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1)
rf.fit(train_X_preprocessed, train_y)

#2-7 (검증 데이터 -> 성능 확인)
from sklearn.metrics import mean_absolute_error
pred_val = rf.predict(valid_X_preprocessed)
print(mean_absolute_error(valid_y, pred_val))

#2-8 (최종 예측)
test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])

#2-9 (저장, 생성)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index = False)
print(pd.read_csv('result.csv').head())

(264, 8) (114, 8) (264,) (114,)
130.73140350877193
      ID    pred
0   4775  156.41
1  10539  357.21
2   8229   89.59
3   8677  263.44
4   5071   60.51


In [132]:
#3-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_1.csv')
#print(df.info())
#print(df.head())

#3-1-1
mean_before = df['before'].mean().round(2)
std_before = round(df['before'].std(), 2)
mean_after = df['after'].mean().round(2)
std_after = round(df['after'].std(), 2)
print(mean_before, std_before, mean_after, std_after)

#3-1-2
from scipy.stats import wilcoxon
statistic, p_value = wilcoxon(df['before'], df['after'])
print(round(statistic, 2))

#3-1-3
p_value = p_value.round(3)
if p_value < 0.05:
  result = '기각'
else:
  result = '채택'

print(result)

#3-2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_2_test.csv')

#print(train.info())
#print(test.info())

#3-1-1
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = 'productivity ~ hours + age + experience'
model = smf.ols(formula, data=train).fit()
#print(model.summary())
coef = model.params[1:].idxmax()
print(coef)

#3-1-2
p_value = model.pvalues[1:]
p_value_sig = p_value[p_value < 0.05]
print(len(p_value_sig))

#3-1-3
'''
model2 = smf.ols(formula, data=test).fit()
r_squared = model2.rsquared
print(r_squared)
'''

test_X = test[['hours', 'age', 'experience']]
test_y = test['productivity']

test_X = sm.add_constant(test_X)

pred_val = model.predict(test_X)
from sklearn.metrics import r2_score
r2 = r2_score(test_y, pred_val)
print(r2)

8.21 1.71 7.23 1.96
72.0
기각
hours
3
0.803642820519101
